In [31]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
crunchbase = pd.read_excel('/Users/mattmullens/Desktop/Python/Crunchbase/crunchbase_export.xlsx'
                   , sheetname=2)

In [3]:
crunchbase['announced_on'] = crunchbase['announced_on'].astype('datetime64')
crunchbase['Year'] = crunchbase['announced_on'].map(lambda x: x.year)
crunchbase['Month'] = crunchbase['announced_on'].map(lambda x: x.month)

In [4]:
crunchbase = crunchbase[crunchbase.announced_on > '04-01-2015']

In [5]:
crunchbase.dtypes

company_name                           object
country_code                           object
state_code                             object
region                                 object
city                                   object
company_category_list                  object
funding_round_type                     object
funding_round_code                     object
announced_on                   datetime64[ns]
raised_amount_usd                     float64
raised_amount                         float64
raised_amount_currency_code            object
investor_count                          int64
investor_names                         object
cb_url                                 object
company_uuid                           object
funding_round_uuid                     object
funding_round_uuid.1                  float64
Year                                    int64
Month                                   int64
dtype: object

In [6]:
capiq = pd.read_excel('/Users/mattmullens/Desktop/Capiq/CapIQ.xls'
                   , sheetname=0 )

In [7]:
capiq.columns=['announced_on','company_name','ticker','fuding_type','funding_status','raised_amount_usd','investor_names','sellers','funding_round_uuid','location','companyid','SICcode','revenue']

In [8]:
capiq['Year'] = capiq['announced_on'].map(lambda x: x.year)
capiq['Month'] = capiq['announced_on'].map(lambda x: x.month)

In [9]:
capiq['company_name'] = capiq['company_name'].str.replace(r"\(.*\)","")
capiq['company_name'] = capiq['company_name'].str.replace( ', Inc.',"")
capiq['company_name'] = capiq['company_name'].str.replace( ', LLC',"")
capiq['company_name'] = capiq['company_name'].str.replace( 'Inc.',"")
capiq['company_name'] = capiq['company_name'].str.replace( 'Corp.',"")
capiq['company_name'] = capiq['company_name'].str.replace( 'Corporation',"")



In [10]:
crunchbase['company_name'] = crunchbase['company_name'].str.replace(r"\(.*\)","")
crunchbase['company_name'] = crunchbase['company_name'].str.replace( ', Inc.',"")
crunchbase['company_name'] = crunchbase['company_name'].str.replace( ', LLC',"")
crunchbase['company_name'] = crunchbase['company_name'].str.replace( 'Inc.',"")
crunchbase['company_name'] = crunchbase['company_name'].str.replace( 'Corp.',"")
crunchbase['company_name'] = crunchbase['company_name'].str.replace( 'Corporation',"")



In [11]:
capiq['company_name'] = capiq['company_name'].apply(lambda x: x.strip())

capiq['company_name'] = capiq['company_name'].apply(lambda x: x.upper())

In [17]:
crunchbasestr = crunchbase.company_name.apply(pd.Series,1).stack()
crunchbasestr.index = crunchbasestr.index.droplevel(-1)

crunchbasestr = crunchbasestr.to_frame()
crunchbasestr.columns=['company_name_str']

In [18]:
crunchbasestr.tail()

,company_name_str
17850,WATCHMASTER
17851,Whitman College
17852,Wikipad
17853,Wytec International
17854,ZappRx


In [19]:
crunchbase = pd.merge(crunchbase, crunchbasestr, left_index=True, right_index=True)

In [20]:
crunchbase['company_name_str'] = crunchbase['company_name_str'].apply(lambda x: x.strip())

In [21]:
crunchbase['company_name_str'] = crunchbase['company_name_str'].apply(lambda x: x.upper())

In [22]:
crunchbase.head()

,company_name,country_code,state_code,region,city,company_category_list,funding_round_type,funding_round_code,announced_on,raised_amount_usd,...,raised_amount_currency_code,investor_count,investor_names,cb_url,company_uuid,funding_round_uuid,funding_round_uuid.1,Year,Month,company_name_str
0,16Lab,JPN,NaN,JPN - Other,Kamakura,Wearables,seed,NaN,2016-03-28,NaN,...,NaN,0,NaN,https://www.crunchbase.com/organization/16lab-...,9454ad28-74da-f461-aecf-c351def34e47,3736b178-1908-a89a-7bb5-853ddb9a4261,NaN,2016,3,16LAB
1,Alientech.com,NaN,NaN,NaN,NaN,NaN,seed,NaN,2016-03-28,NaN,...,NaN,1,Dmitry Fedotov,https://www.crunchbase.com/organization/alient...,83aa5627-5349-aed1-93a1-aa37961aaf11,f7ffbead-7a2c-439f-4481-a5ff90ae879a,NaN,2016,3,ALIENTECH.COM
2,CYBORG,NaN,NaN,NaN,NaN,NaN,seed,NaN,2016-03-28,30000.0,...,NaN,2,"Nokia, PZ Cussons",https://www.crunchbase.com/organization/cyborg,2ae07caa-da3e-6ef7-79d2-d0910878c34e,46af2e2b-467d-77b1-a203-01dafcdcad75,NaN,2016,3,CYBORG
3,droneSim Pro,USA,MD,MD - Other,Poolesville,NaN,angel,NaN,2016-03-28,NaN,...,NaN,0,NaN,https://www.crunchbase.com/organization/drones...,737af71c-7b85-073b-873d-af44c8ff569b,476938de-22d9-3c64-ffdc-c7fcf6be9365,NaN,2016,3,DRONESIM PRO
4,ELUXR,USA,NY,New York City,New York,B2B|E-Commerce Platforms|Jewelry,seed,NaN,2016-03-28,NaN,...,NaN,0,NaN,https://www.crunchbase.com/organization/eluxr-inc,c632bb36-f883-375b-4deb-eba3399b6e3c,e6480ec2-09ff-acc1-05ae-7eb49b850adc,NaN,2016,3,ELUXR


In [23]:
capiq.head(100)

,announced_on,company_name,ticker,fuding_type,funding_status,raised_amount_usd,investor_names,sellers,funding_round_uuid,location,companyid,SICcode,revenue,Year,Month
0,2016-04-18,KINGSTONE COMPANIES,NasdaqCM:KINS,Private Placement,Closed,5,RenaissanceRe Ventures Ltd.,-,IQTR331321807,United States and Canada (Primary),IQ398246,"6331 Fire, marine, and casualty insurance; 641...",63.8,2016,4
1,2016-04-18,MGX MINERALS,DB:1MG,Private Placement,Announced,0.39,-,-,IQTR331322449,United States and Canada (Primary),IQ243716204,-,0,2016,4
2,2016-04-18,PACIFIC NORTH WEST CAPITAL,TSXV:PFN,Private Placement,Announced,0.585,-,-,IQTR331324994,United States and Canada (Primary),IQ3175133,1090 Miscellaneous metal ores,0,2016,4
3,2016-04-18,PROMETHEON PHARMA,-,Private Placement,Announced,2,-,-,IQTR331327411,United States and Canada (Primary),IQ247789087,-,-,2016,4
4,2016-04-17,INNOVATIONAL FUNDING,-,Private Placement,Closed,0.73,Columbia Property Capital,-,IQTR331295276,United States and Canada (Primary),IQ268218294,-,-,2016,4
5,2016-04-17,TRANSACTIS,-,Private Placement,Closed,30,"Safeguard Scientifics, Inc. (NYSE:SFE); The To...",-,IQTR331300099,United States and Canada (Primary),IQ25762246,7371 Computer programming services,10.6,2016,4
6,2016-04-15,INTERNATIONAL MARITIME SECURITY ASSOCIATES,-,Private Placement,Announced,0.25,-,-,IQTR331185624,United States and Canada (Primary),IQ331185377,-,-,2016,4
7,2016-04-15,WAMCO TECHNOLOGY GROUP LTD.,-,Private Placement,Closed,0.012,-,-,IQTR331301094,United States and Canada (Primary),IQ3188692,-,0,2016,4
8,2016-04-15,HELIX BIOPHARMA,TSX:HBP,Private Placement,Announced,3.66,-,-,IQTR331301330,United States and Canada (Primary),IQ876399,2834 Pharmaceutical preparations; 8731 Commerc...,0,2016,4
9,2016-04-15,PROTEUS DIGITAL HEALTH,-,Private Placement,Closed,50,-,-,IQTR331303450,United States and Canada (Primary),IQ8461730,2836 Biological products except diagnostic,46.2,2016,4


In [24]:
len(crunchbase)

17854

In [25]:
len(capiq)

16995

In [34]:
Merged = pd.merge(crunchbase,capiq, left_on=['company_name_str','Year','Month'], right_on=['company_name','Year','Month'])

In [35]:
len(Merged)

2531

In [28]:
Merged[['company_name_x','country_code','state_code','region','city','location','announced_on_x','announced_on_y','raised_amount_usd_x','raised_amount_usd_y','investor_names_x','investor_names_y']][('2015-05-01'> Merged.announced_on_x) & (Merged.announced_on_x> '2015-01-01')].sort_values('company_name_x').head(35)

,company_name_x,country_code,state_code,region,city,location,announced_on_x,announced_on_y,raised_amount_usd_x,raised_amount_usd_y,investor_names_x,investor_names_y
2490,Access Integrated Healthcare,USA,CA,Los Angeles,Los Angeles,United States and Canada (Primary),2015-04-24,2015-04-24,8500000.0,8.5,Penta Mezzanine Fund,Penta Mezzanine Fund
2491,Airviz,USA,PA,Pittsburgh,Pittsburgh,United States and Canada (Primary),2015-04-24,2015-04-24,2000000.0,2.03,Infosys,"Infosys Ltd., Investment Arm"
2472,Albeado,USA,CA,SF Bay Area,Saratoga,United States and Canada (Primary),2015-04-28,2015-04-28,3000000.0,3,Brillio,Brillio
2465,Alpha Moda Lab,USA,CA,SF Bay Area,Palo Alto,United States and Canada (Primary),2015-04-29,2015-04-29,NaN,-,"Flint Capital, Tallwave Capital",Flint Management
2508,Ariste Medical,USA,TN,Memphis,Memphis,United States and Canada (Primary),2015-04-21,2015-04-21,4600000.0,4.6,NaN,-
2524,Bandura,USA,MO,MO - Other,Lake Saint Louis,United States and Canada (Primary),2015-04-14,2015-04-28,NaN,0.552,SixThirty - FinTech Accelerator,-
2455,Bebaio,USA,MA,Boston,Boston,United States and Canada (Primary),2015-04-30,2015-04-30,NaN,1.5,NaN,-
2511,Campus Steps,USA,CA,Los Angeles,Santa Monica,United States and Canada (Primary),2015-04-21,2015-04-21,2000000.0,3,NaN,"OCA Venture Partners, LLC; Rincon Venture Part..."
2510,Campus Steps,USA,CA,Los Angeles,Santa Monica,United States and Canada (Primary),2015-04-21,2015-04-21,1000000.0,3,"OCA Ventures, Rincon Venture Partners, Vicente...","OCA Venture Partners, LLC; Rincon Venture Part..."
2474,CargoSense,USA,VA,"Washington, D.C.",Reston,United States and Canada (Primary),2015-04-28,2015-04-28,2500000.0,3.67,"IrishAngels, Middleburg Capital Development","CIT GAP Funds; IrishAngels, Inc.; Middleburg C..."


In [23]:
Merged.dtypes

company_name_x                         object
country_code                           object
state_code                             object
region                                 object
city                                   object
company_category_list                  object
funding_round_type                     object
funding_round_code                     object
announced_on_x                 datetime64[ns]
raised_amount_usd_x                   float64
raised_amount                         float64
raised_amount_currency_code            object
investor_count                          int64
investor_names_x                       object
cb_url                                 object
company_uuid                           object
funding_round_uuid_x                   object
funding_round_uuid.1                  float64
Year                                    int64
Month                                   int64
company_name_str                       object
announced_on_y                 dat